In [4]:
#### Rama projektowa - Opisywanie obrazów z wykorzystaniem głębokiego uczenia
#### autorzy: Anna Konieczna, Mateusz Ostrowski

### Rama projektowa
Tutorial: https://towardsdatascience.com/image-captioning-with-keras-teaching-computers-to-describe-pictures-c88a46a311b8 <br>
Zbiór danych: <br>
#### Opis:  <br>


In [5]:
#### Config
config = { "images_path": "../Flickr8k_Dataset/",
           "train_images_path": "../Flickr8k_text/Flickr_8k.trainImages.txt", # file conatains the names of images to be used in train data
           "test_images_path": "../Flickr8k_text/Flickr_8k.testImages.txt",
           "train_path": "../Flickr8k_text/Flickr_8k.trainImages.txt", 
           "token_path": "../Flickr8k_text/Flickr8k.token.txt",
           "word_embedings_path": "../glove/glove.6B.200d.txt",
           "embedings_dim": 200, # Polish - 100
           "train_model": True, # True if you intend to train the model,
           "save_model": True,
           "encode_images": False,
           "pretrained_model_path": "",
           "encoded_images_test": "../Pickle/encoded_test_images.pkl",
           "encoded_images_train": "../Pickle/encoded_train_images.pkl",
           "report_path": "",
           "preprocessed_descriptions_save_path": "descriptions.txt",
           "use_lemma": False,
           "decay_lr": False,
           "spacy_lemma_model": "pl_spacy_model",
           "lstm_model_save_dir": "./models/",
           "lstm_model_save_path": "./models/model_dlr_ds_lstm.h5"}

In [6]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from keras import callbacks
from keras import optimizers

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.16.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [7]:
# Load the description set

def load_doc(filename):
	file = open(filename, 'r')
	text = file.read()
	file.close()
	return text

filename = config["token_path"]
# załaduj opisy
doc = load_doc(filename)
print(doc[:300])

1000268201_693b08cb0e.jpg#0	A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg#1	A girl going into a wooden building .
1000268201_693b08cb0e.jpg#2	A little girl climbing into a wooden playhouse .
1000268201_693b08cb0e.jpg#3	A little girl climbing the s


In [8]:
# Load the descriptions to dict structure

def load_descriptions(doc):
	mapping = dict()
	for line in doc.split('\n'):
		tokens = line.split()
		if len(line) < 2:
			continue
		# take the first token as the image id, the rest as the description
		image_id, image_desc = tokens[0], tokens[1:]
		# extract filename from image id
		image_id = image_id.split('.')[0]
		# convert description tokens back to string
		image_desc = ' '.join(image_desc)
		# create the list if needed
		if image_id not in mapping:
			mapping[image_id] = list()
		# store description
		mapping[image_id].append(image_desc)
	return mapping

# parse descriptions
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))

Loaded: 8092 


In [9]:
if config["use_lemma"]:
    import spacy
    nlp = spacy.load('pl_spacy_model')

def clean_descriptions(descriptions):
  # prepare translation table for removing punctuation
  table = str.maketrans('', '', string.punctuation)
  for key, desc_list in descriptions.items():
    for i in range(len(desc_list)):
      desc = desc_list[i]
      if config["use_lemma"]:
          doc = nlp(desc)
      # tokenize
      desc = desc.split()
      # convert to lower case
      desc = [word.lower() for word in desc]
# Lematyzacja
#### spacy model from here: http://zil.ipipan.waw.pl/SpacyPL?action=AttachFile&do=view&target=pl_spacy_model-0.1.0.tar.gz
#### spacy dokumentacja - https://spacy.io/usage
      if config["use_lemma"]:
          desc = [word.lemma_ for word in doc]
      # remove punctuation from each token
      desc = [w.translate(table) for w in desc]
      # remove tokens with numbers in them
      desc = [word for word in desc if word.isalpha()]
      # store as string
      desc_list[i] =  ' '.join(desc)

# clean descriptions
clean_descriptions(descriptions)

In [10]:
# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
	# build a list of all description strings
	all_desc = set()
	for key in descriptions.keys():
		[all_desc.update(d.split()) for d in descriptions[key]]
	return all_desc

# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Original Vocabulary Size: %d' % len(vocabulary))

Original Vocabulary Size: 8775


In [11]:
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

save_descriptions(descriptions, config["preprocessed_descriptions_save_path"])

In [12]:
# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

# load training dataset (6K)
filename = config["train_path"]
train = load_set(filename)
print('Dataset: %d' % len(train))

Dataset: 6000


In [13]:
# Below path contains all the images
images = config["images_path"]
# Create a list of all image names in the directory
img = glob.glob(images + '*.jpg')

In [14]:
# Read the train image names in a set
train_images = set(open(config["train_images_path"], 'r').read().strip().split('\n'))

# Create a list of all the training images with their full path names
train_img = []

for i in img: # img is list of full path names of all images
    if i[len(images):] in train_images: # Check if the image belongs to training set
        train_img.append(i) # Add it to the list of train images

In [15]:
# Below file conatains the names of images to be used in test data
test_images_file = config["test_images_path"]
# Read the validation image names in a set# Read the test image names in a set
test_images = set(open(test_images_file, 'r').read().strip().split('\n'))

# Create a list of all the test images with their full path names
test_img = []

for i in img: # img is list of full path names of all images
    if i[len(images):] in test_images: # Check if the image belongs to test set
        test_img.append(i) # Add it to the list of test images

In [16]:
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# descriptions
train_descriptions = load_clean_descriptions(config["preprocessed_descriptions_save_path"], train)
print('Descriptions: train=%d' % len(train_descriptions))

Descriptions: train=6000


In [17]:
def preprocess(image_path):
    # Convert all the images to size 299x299 as expected by the inception v3 model
    img = image.load_img(image_path, target_size=(299, 299))
    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    return x

In [18]:
# Load the inception v3 model
model = InceptionV3(weights='imagenet')

In [19]:
# Create a new model, by removing the last layer (output layer) from the inception v3
model_new = Model(model.input, model.layers[-2].output)

In [20]:
# Function to encode a given image into a vector of size (2048, )
def encode(image):
    image = preprocess(image) # preprocess the image
    fea_vec = model_new.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec

In [21]:
# Call the funtion to encode all the train images
# This will take a while on CPU - Execute this only once
if config["encode_images"]:
    start = time()
    encoding_train = {}
    for img in train_img:
        encoding_train[img[len(images):]] = encode(img)
        print('.', end='')
    print("Time taken in seconds =", time()-start)

In [22]:
import os
import pickle

if config["encode_images"]:
    mode = 'a' if os.path.exists(config["encoded_images_train"]) else 'wb'

    # Save the bottleneck train features to disk
    with open(config["encoded_images_train"], "wb") as encoded_pickle:
        pickle.dump(encoding_train, encoded_pickle)

In [23]:
# Call the funtion to encode all the test images - Execute this only once
if config["encode_images"]:
    start = time()
    encoding_test = {}
    for img in test_img:
        encoding_test[img[len(images):]] = encode(img)
        print('.', end='')
    print("Time taken in seconds =", time()-start)

In [24]:
# Save the bottleneck test features to disk
if config["encode_images"]:
    with open(config["encoded_images_test"], "wb") as encoded_pickle:
        pickle.dump(encoding_test, encoded_pickle)

In [25]:
with open(config["encoded_images_train"], "rb") as encoded_pickle:
    encoding_train = load(encoded_pickle)
with open(config["encoded_images_test"], "rb") as encoded_pickle:
    encoding_test = load(encoded_pickle)

In [26]:
train_features = load(open(config["encoded_images_train"], "rb"))
print('Photos: train=%d' % len(train_features))

Photos: train=6000


In [27]:
# Create a list of all the training captions
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)
len(all_train_captions)

30000

In [28]:
# Consider only words which occur at least 10 times in the corpus
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
print('preprocessed words %d -> %d' % (len(word_counts), len(vocab)))

preprocessed words 7589 -> 1654


In [29]:
ixtoword = {}
wordtoix = {}

ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

In [30]:
vocab_size = len(ixtoword) + 1 # one for appended 0's
vocab_size

1655

In [31]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

Description Length: 37


In [32]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key+'.jpg']
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
#                 yield [[array(X1), array(X2)], array(y)]
                yield ([array(X1), array(X2)], array(y)) 
                X1, X2, y = list(), list(), list()
                n=0

In [33]:
# Load Glove vectors

embeddings_index = {} # empty dictionary

f = open(config["word_embedings_path"], encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [34]:
# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, config["embedings_dim"]))

for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [35]:
embedding_matrix.shape

(1655, 200)

In [36]:
# Deep stacked LSTM
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, config["embedings_dim"], mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256, return_sequences=True, input_shape=(2048,))(se2)
se4 = LSTM(256, return_sequences=True)(se3)
se5 = LSTM(256)(se4)
decoder1 = add([fe2, se5])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [37]:
# inputs1 = Input(shape=(2048,))
# fe1 = Dropout(0.5)(inputs1)
# fe2 = Dense(256, activation='relu')(fe1)
# inputs2 = Input(shape=(max_length,))
# se1 = Embedding(vocab_size, config["embedings_dim"], mask_zero=True)(inputs2)
# se2 = Dropout(0.5)(se1)
# se3 = LSTM(256)(se2)

# decoder1 = add([fe2, se3])
# decoder2 = Dense(256, activation='relu')(decoder1)
# outputs = Dense(vocab_size, activation='softmax')(decoder2)
# model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [38]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 37)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 37, 200)      331000      input_3[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 37, 200)      0           embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2048)]       0                                            
_______________________________________________________________________________________

In [39]:
model.layers[1]

In [40]:
model.layers[1].set_weights([embedding_matrix])
model.layers[1].trainable = False

In [41]:
if config["decay_lr"]:
    opt = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
else:    
    opt = optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt)

In [42]:
if config["train_model"]:
    epochs = 100
    number_pics_per_bath = 3
    steps = len(train_descriptions)//number_pics_per_bath

In [ ]:
if config["train_model"]:
    callback = callbacks.EarlyStopping(monitor='loss',min_delta = 0.001, patience=2)
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    model.fit_generator(generator, epochs=epochs, steps_per_epoch=steps, callbacks=[callback], verbose=1)
    if config["save_model"]:
        writepath = config["lstm_model_save_dir"] + 'model_' + str(i) + '.h5'
        mode = 'a' if os.path.exists(writepath) else 'wb'
        model.save(writepath)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
1283/2000 [==================>...........] - ETA: 4:15 - loss: 4.0356

In [ ]:
if config["train_model"]:
    #model.optimizer.lr = 0.0001
    epochs = 100
    number_pics_per_bath = 6
    steps = len(train_descriptions)//number_pics_per_bath

In [ ]:
if config["train_model"]:
#     for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    model.fit_generator(generator, epochs=epochs, steps_per_epoch=steps, callbacks=[callback], verbose=1)
    #model.save('./model_weights/model_' + str(i) + '.h5')

In [ ]:
if config["save_model"]:
    writepath = config["lstm_model_save_dir"] + 'model_' + str(i) + '.h5'
    mode = 'a' if os.path.exists(writepath) else 'wb'
    model.save(writepath)

In [ ]:
images = config["images_path"]

In [ ]:
with open(config["encoded_images_test"], "rb") as encoded_pickle:
    encoding_test = load(encoded_pickle)

In [ ]:
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [ ]:
for i in range(5, 10):
    pic = list(encoding_test.keys())[i]
    image = encoding_test[pic].reshape((1,2048))
    x=plt.imread(images+pic)
    plt.imshow(x)
    plt.show()
    print("Greedy:",greedySearch(image))
    print(descriptions[pic[:-4]])

In [ ]:
#### Metrics
##### Bag of words = bow

def get_bow(description):
    bow = description.split(' ')
    bow = set(bow)
    return bow

#calculate bow score
def bow_score(bow_train, bow_pred):
    overlap = bow_train & bow_pred
    universe = bow_train | bow_pred

    result1 = float(len(overlap)) / len(bow_train) * 100
    result2 = float(len(overlap)) / len(bow_pred) * 100
    result3 = float(len(overlap)) / len(universe) * 100
    return result1, len(overlap)

In [ ]:

##### Cider
#function for calculating cider metric
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def cider(given_captions, prediction):
    prediction_arr = []
    prediction_arr.append(prediction)
    vectorizer = TfidfVectorizer()
    trainedV = vectorizer.fit(given_captions)
    givenTfIdf = trainedV.transform(given_captions).toarray()
    predTfIdf = trainedV.transform(prediction_arr).toarray()
    partialCiders = 0
    help_arr = []
    
    for c in givenTfIdf:
        help_arr.append(c)
        partialCiders = partialCiders + cosine_similarity(predTfIdf,help_arr)
        help_arr = []
        
    return partialCiders/len(givenTfIdf)

In [ ]:
#### 
import warnings; warnings.simplefilter('ignore')
from IPython.display import clear_output

#preparation of metrics lists
bleu_train_scores = []
meteor_train_scores = []
cider_train_scores = []
    
# for i in range(0,5):
#     bleu_train_scores.append([])
#     meteor_train_scores.append([])
#     cider_train_scores.append([])
    
train_pics = list(encoding_train.keys())

#calculation of metrics for train images dataset
for j in range(0,len(train_pics)):
    pic = train_pics[j]
    image = encoding_train[pic].reshape((1,2048))

    actual_desc_0 = descriptions[pic.rsplit( ".", 1 )[ 0 ]][0]
    actual_desc_1 = descriptions[pic.rsplit( ".", 1 )[ 0 ]][1]
    actual_desc_2 = descriptions[pic.rsplit( ".", 1 )[ 0 ]][2]
    actual_desc_3 = descriptions[pic.rsplit( ".", 1 )[ 0 ]][3]
    actual_desc_4 = descriptions[pic.rsplit( ".", 1 )[ 0 ]][4]

    reference = [actual_desc_0.split(),
         actual_desc_1.split(),
         actual_desc_2.split(),
         actual_desc_3.split(),
         actual_desc_4.split()]

    generated = greedySearch(image)
    bleu_train_scores.append(sentence_bleu(reference, generated.split(), weights=(1, 0, 0, 0)))
    meteor_train_scores.append(meteor_score(descriptions[pic[:-4]],generated))
    cider_train_scores.append(cider(descriptions[pic[:-4]],generated)[0][0])

    clear_output(wait=True)
    print('train',j,len(train_pics), flush=True)



In [ ]:
from numpy import mean

result_data = {}

#calculate mean score based on socres list for each metric

result_data = [
    mean(bleu_train_scores),
    mean(meteor_train_scores),
    mean(cider_train_scores),
]

print(['Bleu TREN', 'Meteor TREN','CIDEr TREN'])
result_data

In [ ]:
from tqdm.notebook import tqdm

#preparation of metrics lists
bleu_test_scores = []
meteor_test_scores = []
cider_test_scores = []
test_pics = list(encoding_test.keys())

#calculation of metrics for test images dataset
for j in tqdm(range(0,len(test_pics))):
    pic = test_pics[j]
    image = encoding_test[pic].reshape((1,2048))

    actual_desc_0 = descriptions[pic.rsplit( ".", 1 )[ 0 ]][0]
    actual_desc_1 = descriptions[pic.rsplit( ".", 1 )[ 0 ]][1]
    actual_desc_2 = descriptions[pic.rsplit( ".", 1 )[ 0 ]][2]
    actual_desc_3 = descriptions[pic.rsplit( ".", 1 )[ 0 ]][3]
    actual_desc_4 = descriptions[pic.rsplit( ".", 1 )[ 0 ]][4]

    reference = [actual_desc_0.split(),
                 actual_desc_1.split(),
                 actual_desc_2.split(),
                 actual_desc_3.split(),
                 actual_desc_4.split()]

    generated = greedySearch(image)

    bleu_test_scores.append(sentence_bleu(reference, generated.split(), weights=(1, 0, 0, 0)))
    meteor_test_scores.append(meteor_score(descriptions[pic[:-4]],generated))
    cider_test_scores.append(cider(descriptions[pic[:-4]],generated)[0][0])

    clear_output(wait=True)
    print('test', j, len(test_pics), flush=True)



In [ ]:
#calculate mean score based on socres list for each metric
from numpy import mean

result_data = {}

# for i in range(0,5):
result_data = [
    mean(bleu_test_scores),
    mean(meteor_test_scores),
    mean(cider_test_scores)
]

print(['Bleu TEST', 'Meteor TEST', 'CIDEr TEST'])
result_data

Early stopping patience=3, delta = 0.001, bleu weights=(0.25, 0.25, 0.25, 0.25)

['Bleu TREN', 'Meteor TREN', 'CIDEr TREN']
[0.08826357992794069, 0.3812428392811727, 0.2538227205501579]

['Bleu TEST', 'Meteor TEST', 'CIDEr TEST']
[0.06709583754955759, 0.35676080070908517, 0.24570790686788618]


Early stopping patience=3, delta = 0.001,
['Bleu TEST', 'Meteor TEST', 'CIDEr TEST']
[0.5314928729248558, 0.35676080070908517, 0.24570790686788618]

Decaying learning rate
['Bleu TREN', 'Meteor TREN', 'CIDEr TREN']
[0.5389221988318406, 0.36500372248553375, 0.246789750564132]<>
['Bleu TEST', 'Meteor TEST', 'CIDEr TEST']
[0.5099813205752186, 0.3409624319787307, 0.23597343552878425]

Przy overfittingu wygenerowane zdania są dłuższe/bardziej złożone, lecz mniej wiernie opisują obrazek. Np.:
reedy: a man in a black shirt is sitting on a bench in front of a brick building
['a japanese swordsman jumps into the air', 'a man in a blue outfit is swinging a sword in the air', 'an actor dressed as a pirate performs in an outdoor park', 'a person wearing a blue robe is in the air holding a sword', 'man in martial artsstyle costume jumping in midair']

Na obrazku nie było ani ławki ani budynku.

### Deep Stacked LSTM + decaying learning rate
['Bleu TEST', 'Meteor TEST', 'CIDEr TEST']
[0.4791308028567803, 0.31819220287157657, 0.22750624379012033]

### Early stopping DS-LSTM
['Bleu TEST', 'Meteor TEST', 'CIDEr TEST']
[0.4811540307541167, 0.32533699474459354, 0.22786212254431967]
['Bleu TREN', 'Meteor TREN', 'CIDEr TREN']
[0.5059629215074034, 0.3502870455324523, 0.23924489269523558]